## This notebook demonstrates how to perform bulk inference of [Phi-3-mini-128k-instruct](https://huggingface.co/microsoft/Phi-3-mini-128k-instruct) on the [Tracto.ai](https://tracto.ai/) platform.

Tracto is perfect for offline batch inference:

* Easy scaling - just change the `job_count` parameter.
* Fault tolerance out of the box - if a job crashes, it gets restarted automatically. No need to handle GPU failures or infrastructure issues - Tracto takes care of it.
* Full Tracto integration - save results directly to the distributed file system and process them further on the platform.

In [1]:
# configure environment to run this notebooks
import uuid
import yt.wrapper as yt

username = yt.get_user_name()
if yt.exists(f"//sys/users/{username}/@user_info/home_path"):
    # prepare working directory on distributed file system
    user_info = yt.get(f"//sys/users/{yt.get_user_name()}/@user_info")
    homedir = user_info["home_path"]
    # find avaliable vm presets
    cpu_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("cpu")] or ["default"]
    h100_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("h100")]
    h100_8_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("h100-8")]
    workdir = f"{homedir}/tmp/demo_workdir/{uuid.uuid4().hex}"
else:
    cpu_pool_trees = ["default"]
    h100_pool_trees = ["gpu_h100"]
    h100_8_pool_trees = ["gpu_h100"]
    workdir = f"//tmp/examples/{uuid.uuid4().hex}"

yt.create("map_node", workdir, recursive=True, ignore_existing=True)
print("Current working directory:", workdir)

Current working directory: //home/equal_amethyst_vulture/tmp/demo_workdir/cd73f567cb5f449cacd18daa1f1b5947


Prepare data for inference as a table.

In [3]:
from datasets import load_dataset

dataset = load_dataset("Rapidata/Other-Animals-10")

table_path = f"{workdir}/questions"
yt.create("table", table_path, force=True)

questions = [
    {"question": f"Can a {animal} fly? If it can't - how can we help it take off?"}
    for animal in set(dataset["train"].features["label"].int2str(dataset["train"]["label"]))
]

yt.write_table(table_path, questions)

Run bulk inference of `Phi-3-mini`.

In [5]:
from typing import Iterable
import logging
import sys
import random

yt.config["pickling"]["safe_stream_mode"] = False


result_path = f"{workdir}/result"


@yt.aggregator
def bulk_inference(records: Iterable[dict[str, str]]) -> dict[str, str]:
    from vllm import LLM, SamplingParams

    llm = LLM(model="microsoft/Phi-3-mini-128k-instruct", trust_remote_code=True)
    sampling_params = SamplingParams(
        temperature=0.6,
        top_p=0.9,
        max_tokens=32000,
    )

    conversations = [
        [
            {
                "role": "user",
                "content": record["question"],
            },
        ]
        for record in records
    ]
    outputs = llm.chat(
        messages=conversations,
        sampling_params=sampling_params,
    )
    for prompt, output in zip(conversations, outputs):
        yield {
            "prompt": prompt,
            "text": output.outputs[0].text,
        }


yt.run_map(
    bulk_inference,
    table_path,
    result_path,
    job_count=2,
    spec={
        "pool_trees": h100_pool_trees,
        "mapper": {
            "gpu_limit": 1,
            "memory_limit": 32212254720,
            "cpu_limit": 20,
        },
    },
)

2025-06-19 18:42:15,530	INFO	Operation started: https://playground.tracto.ai/playground/operations/a1763b10-17e4dba0-24dd03e8-90e0819f/details


2025-06-19 18:42:15,574	INFO	( 0 min) operation a1763b10-17e4dba0-24dd03e8-90e0819f starting


2025-06-19 18:42:16,124	INFO	( 0 min) operation a1763b10-17e4dba0-24dd03e8-90e0819f initializing


2025-06-19 18:42:17,819	INFO	( 0 min) Unrecognized spec: {'enable_partitioned_data_balancing': false, 'mapper': {'title': 'bulk_inference'}}


2025-06-19 18:42:17,862	INFO	( 0 min) operation a1763b10-17e4dba0-24dd03e8-90e0819f: running=0     completed=0     pending=2     failed=0     aborted=0     lost=0     total=2     blocked=0    


2025-06-19 18:42:23,865	INFO	( 0 min) operation a1763b10-17e4dba0-24dd03e8-90e0819f: running=1     completed=0     pending=1     failed=0     aborted=0     lost=0     total=2     blocked=0    


2025-06-19 18:44:17,768	INFO	( 2 min) operation a1763b10-17e4dba0-24dd03e8-90e0819f: running=1     completed=1     pending=0     failed=0     aborted=0     lost=0     total=2     blocked=0    


2025-06-19 18:45:54,327	INFO	( 3 min) operation a1763b10-17e4dba0-24dd03e8-90e0819f completed


In [6]:
for record in yt.read_table(result_path):
    print(f"Q: {record['prompt']}")
    print(f"A: {record['text']}")

Q: [{'role': 'user', 'content': "Can a calf fly? If it can't - how can we help it take off?"}]
A:  No, a calf cannot fly. Calves are young cattle, and like all terrestrial mammals, they are not equipped with the necessary adaptations for flight. They have solid bones, no wings, and their body structure is not designed to generate lift or propulsion through the air.

However, if you're asking about helping a calf to take off in a playful or imaginative context (e.g., a game or a story), you could consider the following ideas:

1. Make it a game of imagination: Encourage children to use their creativity and play with the idea of flying, using props like wings or helium balloons.

2. Use a toy or costume: Give the calf a toy or costume with wings, and have children pretend to help it fly by flapping the wings or blowing on the costume.

3. Create a story or puppet show: Develop a story or puppet show where the calf learns to fly through the power of friendship, determination, or a magical